In [ ]:
pip install transformers

In [ ]:
#sourse = https://discuss.huggingface.co/t/error-with-new-tokenizers-urgent/2847/2

In [32]:
#model from hugging face
from transformers import AutoModelWithLMHead, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-question-generation-ap", use_fast=False)
model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-question-generation-ap")

def get_question(answer, context, max_length=64):
  input_text = "answer: %s  context: %s </s>" % (answer, context)
  features = tokenizer([input_text], return_tensors='pt')

  output = model.generate(input_ids=features['input_ids'],
               attention_mask=features['attention_mask'],
               max_length=max_length)

  return tokenizer.decode(output[0])

context = "Manuel has created RuPERTa-base with the support of HF-Transformers and Google"
answer = "Manuel"

get_question(answer, context)

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1581: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


'<pad> question: Who created the RuPERTa-base?</s>'

In [33]:
#trial
context = "Manipal Institute of Technology is a private engineering school under Manipal Academy of Higher Education in India. The Manipal Institute of Technology, Manipal, was established in 1957 as one of the first self-financing engineering colleges in the country. "
answer = "Manipal Institute of Technology"

get_question(answer, context)

'<pad> question: What is the name of the private engineering school in India?</s>'

In [34]:
#grant permission (for gdrive use only)
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [35]:
cd gdrive/MyDrive/Colab\ Notebooks

[Errno 2] No such file or directory: 'gdrive/MyDrive/Colab Notebooks'
/content/gdrive/MyDrive/Colab Notebooks


In [ ]:
import random
import pandas as np
import numpy as np
from string import whitespace

#isolating context

#parse through file with text content (Note: save file in directory as context.txt)
f = open("context.txt", "r", encoding='utf-8')
content = f.readlines()
full_content = content
mod_content = []

#filter based on new line characters and empty list items
for i in content:
    result = i.splitlines()
    for j in result:
        if j!= '\n' and j!='':
            mod_content.append(j)

#display
for cont in mod_content:
    print(cont)

MIT offers a Bachelor’s and Master’s program in engineering streams. It also gives students the option to carry out full-time or part-time research. The minimum duration for postgraduate qualified PhD students is three years for full-time, and four years for part-time. There is a one-year relaxation for applicants with an MPhil degree. The institute undertakes sponsored research programs supported by funding agencies such as DST, CSIR, AICTE, and the Ministry of Environmental Sciences. It has collaborative research programs in association with premier research laboratories and institutes in India and abroad.
The Manipal Institute of Technology (MIT), Manipal was established in 1957 as one of the first self-financing engineering colleges in the country
Manipal Institute of Technology (MIT) Manipal has always been very proactive in placing students into their preferred profiles. The institute has a well-organized placement team headed by the Associate Director, three Assistant Directors,

In [ ]:
#works for questions associated with larger contexts, do a shorter way with sentence separated context (whole context is answer)
#randomly selecting answer region based on random index generation, does not work as well alterntive 2
qa_dict = {}
for cont in mod_content:
    space_indices = [i for i, char in enumerate(cont) if char in whitespace]
    #print(space_indices)
    q5_questions = []
    for j in range(0, 5):
        a = random.choice(space_indices)
        b_index = space_indices.index(a) + 10
        #print(b_index)
        #print(len(space_indices))
        if(int(b_index/len(space_indices)) != 0):
            b = space_indices[b_index%len(space_indices)]
            answer = cont[b:a + 1]
        else:
            b = space_indices[b_index]
            answer = cont[a:b + 1]
        q = get_question(answer, cont)
        start = q.index(':') + 1
        end = q.rfind('?') + 1
        q = q[start:end]
        print(q)
        if q not in q5_questions:
          q5_questions.append(q)
        """ print("The question is" + q)
        print("The answer is" + answer)
        print("The context is" + cont)
        print()"""
    qa_dict[cont] = q5_questions

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:303: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(


 What is the minimum duration for postgraduate students at MIT?
 What is the maximum duration for a PhD student at MIT?
 What kind of research programs does MIT have?
 What is the relationship between MIT and other research institutes?
 What types of programs does MIT offer?
 When was the Manipal Institute of Technology established?
 What was the name of the first self-financing engineering college in India?
 What was the Manipal Institute of Technology established in?
 What was the name of the first self-financing engineering college in India?
 What is the Manipal Institute of Technology?
 Who heads the placement team at MIT Manipal?
 What has Manipal always been proactive in?
 What has been the focus of the placement process at MIT Manipal?
 What provides seamless support to the students in the placement process?
 Who provides seamless support to the students in the placement process?
 What is the function of the Career Counselling Cell?
 What does MIT have?
 What does the Career Cou

In [ ]:
import json

json_object = json.dumps(qa_dict, indent=4)
with open("data.json", "w") as outfile:
    outfile.write(json_object)


In [36]:
#Alternative: split context by periods generate answers for period separated sentences
cont_ans_dict = {}
for cont in mod_content:
    cont_ans_dict[cont] = cont.split('. ')
print(cont_ans_dict)

qa_dict = {}
for context in cont_ans_dict.keys():
    questions = []
    for ans in cont_ans_dict[context]:
        q = get_question(ans, context)
        q_bad = q
        start = q.index(':') + 1
        end = q.rfind('</s>')
        q = q[start:end]
        # if(q == ''):
        #   print(start)
        #   print(end)
        #   print(q_bad)
        ans_ind = context.find(ans)
        context_squad_ans = context[0: ans_ind - 1] + '<ans>' + context[ans_ind: ans_ind + len(ans) - 1] + '</ans>' + context[ans_ind + len(ans): -1]

        if q not in qa_dict.keys():
          qa_dict[q] = context_squad_ans
          print(q)
          print(context_squad_ans)




{'MIT offers a Bachelor’s and Master’s program in engineering streams. It also gives students the option to carry out full-time or part-time research. The minimum duration for postgraduate qualified PhD students is three years for full-time, and four years for part-time. There is a one-year relaxation for applicants with an MPhil degree. The institute undertakes sponsored research programs supported by funding agencies such as DST, CSIR, AICTE, and the Ministry of Environmental Sciences. It has collaborative research programs in association with premier research laboratories and institutes in India and abroad.': ['MIT offers a Bachelor’s and Master’s program in engineering streams', 'It also gives students the option to carry out full-time or part-time research', 'The minimum duration for postgraduate qualified PhD students is three years for full-time, and four years for part-time', 'There is a one-year relaxation for applicants with an MPhil degree', 'The institute undertakes sponsor

In [38]:
#save dictionary into txt file
import json

with open("squad_style_qa.txt", "w", encoding='utf-8') as fp:
    json.dump(qa_dict, fp, ensure_ascii=False)  # encode dict into JSON
